In [14]:
import json

In [21]:
with open('../json/computing_order.json', 'r') as f:
    computing_order = json.load(f)
    
with open('../json/children_light.json', 'r') as f:
    children_light = json.load(f)

with open('../json/formulas_light.json', 'r') as f:
    formulas_light = json.load(f)
    
with open('../json/constants_light.json', 'r') as f:
    constants_light = json.load(f)
    
with open('../json/inputs_light.json', 'r') as f:
    inputs_light = json.load(f)
    
with open('../json/unknowns_light.json', 'r') as f:
    unknowns_light = json.load(f)
    
with open('../json/input_variables.json', 'r') as f:
    input_variables = json.load(f)


## Function definitions

In [42]:
import math

def product(l):
    accu = 1.
    for e in l:
        accu *= e
    return accu

def boolean_or(l):
    for e in l:
        if l:
            return 1.
    return 0.

def boolean_et(l):
    for e in l:
        if not l:
            return 0.
    return 1.

functions_mapping = {
    '+': sum,
    '*': product,
    '-': (lambda x: -x[0]),
    'positif': (lambda x: float(x[0]>0)),
    'positif_ou_nul': (lambda x: float(x[0]>=0)),
    'null': (lambda x: float(x[0]==0)),
    'operator:>=': (lambda x: float(x[0]>=x[1])),
    'operator:=': (lambda x: float(x[0]==x[1])),
    'ternary': (lambda x: x[1] if x[0] else x[2]),
    'si': (lambda x: x[1] if x[0] else 0.),
    '/': (lambda x: x[0]/x[1] if x[1] else 0.),
    'max': max,
    'min': min,
    'inf': (lambda x: math.floor(x[0])),
    'arr': (lambda x: round(x[0])),
    'abs': (lambda x: abs(x[0])),
    'present': (lambda x: x[0] != 0.),
    'boolean:ou': boolean_or,
    'boolean:et': boolean_et,
    'dans': (lambda x: 1. if (x[0] in x[1:]) else 0.)
}


## Computation

http://www3.finances.gouv.fr/calcul_impot/2015/index.htm


In [30]:
alias_values = {'1AJ': 30000.}

In [31]:
input_values = {}
for var in input_variables:
    name = var['name']
    alias = var['alias']
    
    if alias in alias_values:
        input_values[name] = alias_values[alias]
        
input_values

{'TSHALLOV': 30000.0}

In [32]:
def get_value(name):
    if name in formulas_light:
        return computed_values[name]
    
    if name in constants_light:
        return constants_light[name]
        
    if name in inputs_light:
        if name in input_values:
            return input_values[name]
        return 0.
        
    if name in unknowns_light:
        return 0.
    
    raise Exception('Unknown variable category.')


In [36]:
def compute(node):
    nodetype = node['nodetype']

    if nodetype == 'symbol':
        name = node['name']
        value = get_value(name)
        return value 
        
    if nodetype == 'float':
        value = node['value']
        return value
    
    if nodetype == 'call':
        name = node['name']
        args = [compute(child) for child in node['args']]
        function = functions_mapping[name]
        value = function(args)
        return value
        
    raise ValueError('Unknown type : %s'%nodetype)

In [43]:
computed_values = {}

for variable in computing_order:
    formula = formulas_light[variable]
    computed_values[variable] = compute(formula)


In [44]:
computed_values['IRN']

2461.0